In [ ]:
import holoviews as hv
import geoviews as gv
import cartograph
import numpy as np
from collections import defaultdict
hv.extension('bokeh')
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
br_x = (-8649435.388132825, -3472043.511082467)
br_y = (-4831406.481658398, 744246.3090112011)
max_edges = int(1e4)

In [ ]:
carto = cartograph.Cartography('<PATH HERE>')

In [ ]:
bg_map = gv.tile_sources.CartoLight.opts(xaxis=None, yaxis=None, width=1200, height=600)
width_by_level = {0: 3, 1: 3, 2: 2, 3: 2, 4: 1, 5: 1}
color_by_level = {0: 'orange', 1: 'orange', 2: 'lightblue', 3: 'lightblue', 4: 'gray', 5: 'gray'}

def update_path(x_range, y_range):
    x_range = x_range or br_x
    y_range = y_range or br_y
    edges_by_level = carto.sample_edges((x_range[0], y_range[0]), (x_range[1], y_range[1]), max_edges)
    
    def get_path(level):
        return hv.Path(edges_by_level.get(level, [])).opts(line_width=width_by_level[level], line_color=color_by_level[level])
    
    return get_path(0) * get_path(1) * get_path(2) * get_path(3) * get_path(4) * get_path(5)

hv.DynamicMap(update_path, streams=[hv.streams.RangeXY()]) * bg_map

In [ ]:
scc = carto.strongly_connected_components()

In [ ]:
import pandas as pd
df = pd.DataFrame([{
    'num_nodes': len(s)
} for s in scc])
print(len(df))
df.head()

In [ ]:
count = df.groupby('num_nodes').size().to_frame('num_strongly_connected_sub_graphs').reset_index()
count

In [ ]:
count['num_strongly_connected_nodes'] = count['num_nodes'] * count['num_strongly_connected_sub_graphs']
count['% nodes'] = count['num_strongly_connected_nodes'] / df['num_nodes'].sum()
count.sort_values('% nodes', ascending=False).head()